In [ ]:
# Importieren der erforderlichen Bibliotheken
import random
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
import numpy as np
from pyspark.sql.functions import count, avg, desc, round as round_func
from pyspark.sql.functions import col, count, lit, percentile_approx
from pyspark.sql.functions import trim, regexp_replace
from pyspark.sql.functions import when

In [ ]:
# Spark Session initialisieren
spark = SparkSession.builder \
    .appName("BookCrossing_Angriffsprofile Nuke") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "16g") \
    .getOrCreate()

# Lesen der Ratings-Datei
ratings_df = spark.read.option("header", "true").option("delimiter", ";").csv("BX-Book-Ratings.csv")
ratings_df.show(10)

In [ ]:
# Laden der books.csv
books_df = spark.read.option("header", "true").option("delimiter", ";").csv("BX_Books.csv")
books_df.show(10)

In [ ]:
# Verbinden der DataFrames basierend auf der ISBN
ratings = ratings_df.join(books_df, on='ISBN', how='left')

# Umbenennen der Spalten zu rating und userId
ratings = ratings.withColumnRenamed("Book-Rating", "rating")
ratings = ratings.withColumnRenamed("User-ID", "userId")

# Entfernen der nicht benötigten Spalten
ratings = ratings.drop('Book-Title')
ratings = ratings.drop('Book-Author')
ratings = ratings.drop('Year-Of-Publication')
ratings = ratings.drop('Image-URL-S')
ratings = ratings.drop('Image-URL-M')
ratings = ratings.drop('Image-URL-L')

# Schema und erste 10 Einträge des Datensatzes
ratings.printSchema()
ratings.show(10)

In [ ]:
# Anzahl der Ratings im Datensatz
print("Anzahl der Ratings im Datensatz:", ratings.count())

# Duplikate filtern
duplicates = (
    ratings.groupBy("userId", "ISBN")
    .count()
    .filter(F.col("count") > 1)
)

# Duplikate anzeigen
print("Anzahl der Duplikate:", duplicates.count())

# Duplikate entfernen
ratings = ratings.dropDuplicates(["userId", "ISBN"])

# Ergebnis überprüfen: Anzahl der Ratings im Datensatz nach Entfernung der Duplikate
print("Anzahl der Zeilen nach Entfernen der Duplikate:")
print(ratings.count())

In [ ]:
# Berechne den Mittelwert und die Standardabweichung der Ratings im Datensatz und zeige diese an
mean_rating_system = ratings.agg({"rating": "avg"}).collect()[0][0]
stddev_rating = ratings.agg({"rating": "stddev"}).collect()[0][0]

print(mean_rating_system)
print(stddev_rating)

In [ ]:
# Anzahl aller Ratings im DataFrame
total_ratings_counter = ratings_df.count()

# Ausgabe der Anzahl der Ratings
print("Anzahl aller Ratings:", total_ratings_counter)

In [ ]:
# Berechnung der Beliebtheit und der durchschnittlichen Bewertung pro Buch
item_popularity_and_avg_rating = (
    ratings.groupBy("ISBN")
    .agg(
        F.count("rating").alias("num_ratings"),
        F.avg("rating").alias("avg_rating")
    )
)

# Finde das mittlere Quantil für die Anzahl der Bewertungen
quantiles = item_popularity_and_avg_rating.approxQuantile("num_ratings", [0.4, 0.6], 0.05)
min_num_ratings, max_num_ratings = quantiles[0], quantiles[1]

# Mittlere Bewertung: Finde das mittlere Quantil für die durchschnittliche Bewertung
rating_quantiles = item_popularity_and_avg_rating.approxQuantile("avg_rating", [0.4, 0.6], 0.05)
min_avg_rating, max_avg_rating = rating_quantiles[0], rating_quantiles[1]

# Wähle repräsentativen Publisher
popular_publishers = (
    books_df
    .withColumn("Publisher", F.explode(F.split(F.col("Publisher"), "\|")))
    .groupBy("Publisher")
    .count()
    .orderBy(F.desc("count"))
    .limit(3)
)

# Filtere die Bücher mit mittlerer Beliebtheit, mittlerer Bewertung und einem repräsentativen Publisher
target_candidates = (
    item_popularity_and_avg_rating
    .filter(
        (F.col("num_ratings") >= min_num_ratings) & (F.col("num_ratings") <= max_num_ratings) &
        (F.col("avg_rating") >= min_avg_rating) & (F.col("avg_rating") <= max_avg_rating)
    )
    .join(books_df, "ISBN")
    .withColumn("Publisher", F.explode(F.split(F.col("Publisher"), "\|")))
    .join(popular_publishers, "Publisher")
)

# Zeige ein paar Kandidaten, die als Target-Item verwendet weden könnten
target_candidates.show(5)

In [ ]:
# Filtere den DataFrame so, dass nur ISBNs mit einem nicht-leeren Publisher vorhanden sind
filtered_books = ratings.filter(ratings["Publisher"].isNotNull() & (ratings["Publisher"] != ""))

# Liste aller verfügbaren ISBNs aus dem Datensatz abrufen
all_isbns = ratings.select("ISBN").distinct().rdd.flatMap(lambda x: x).collect()

# Funktion zum Validieren der eingegebenen ISBN
def get_valid_target_item():
    while True:
        user_input = input("Bitte gib die zehnstellige ISBN des Ziel-Items ein oder tippe 'zufällig' für ein zufälliges Item: ")
        
        if user_input.lower() == 'zufällig':
            # Wähle ein zufälliges gültiges Item aus der Liste
            return random.choice(all_isbns)
        
        try:
            # Nutzeraufforderung in eine Zahl umwandeln
            target_item = str(user_input)
            
            # Überprüfen, ob die eingegebene ISBN im Datensatz vorhanden ist
            if target_item in all_isbns:
                return target_item
            else:
                print("Die eingegebene ISBN ist ungültig. Bitte versuche es erneut.")
        except ValueError:
            print("Ungültige Eingabe! Bitte gib eine gültige ISBN ein oder tippe 'zufällig'.")

# Abrufen der validierten ISBN vom Nutzer
target_item = get_valid_target_item()

print(f"Das Ziel-Item (ISBN) für den Angriff lautet: {target_item}")

In [ ]:
# Gesamtanzahl der User Profile im Datensatz
total_user_profiles = ratings.select("userId").distinct().count()

# Aufforderung, einen %-Anteil für die Anzahl der Angriffsprofile einzugeben
percentage_of_attacks = float(input("Bitte gib den Prozentsatz der Angriffsprofile ein (z.B. 5 für 5%): "))

# Berechne die Anzahl der Angriffsprofile basierend auf dem eingegebenen Prozentsatz
num_attack_profiles = int((percentage_of_attacks / 100) * total_user_profiles)

print(f"Die Anzahl der Angriffsprofile beträgt: {num_attack_profiles}")

In [ ]:
# Berechnung der Durchschnittsbewertungen pro ISBN
item_avg_ratings = ratings.groupBy("ISBN").agg(avg("rating").alias("avg_rating"))
item_avg_ratings.show(10)
# Konvertiere item_avg_ratings in ein Dictionary für schnellen Zugriff
item_avg_ratings_dict = {row["ISBN"]: row["avg_rating"] for row in item_avg_ratings.collect()}

In [ ]:
# Bereinige die ISBN-Spalte (entferne Leerzeichen oder Sonderzeichen)
ratings = ratings.withColumn("ISBN", trim(col("ISBN")))
ratings = ratings.withColumn("ISBN", regexp_replace(col("ISBN"), r"[^a-zA-Z0-9]", ""))

item_avg_ratings = item_avg_ratings.withColumn("ISBN", trim(col("ISBN")))
item_avg_ratings = item_avg_ratings.withColumn("ISBN", regexp_replace(col("ISBN"), r"[^a-zA-Z0-9]", ""))

# Berechnung des Schwellenwerts für das untere Quartil der Durchschnittsbewertungen
lower_quartile = item_avg_ratings.approxQuantile("avg_rating", [0.25], 0.01)[0]  # 25. Perzentil

# Filtere Items nach dem unteren Quartil und sortiere nach Anzahl der Bewertungen
unpopular_items_negative = ratings.join(item_avg_ratings, "ISBN") \
    .filter((col("avg_rating") <= lit(lower_quartile))) \
    .groupBy("ISBN") \
    .agg(count("*").alias("rating_count")) \
    .orderBy(col("rating_count").asc()) \
    .limit(2)

# Konvertiere das Ergebnis in eine Liste, wobei das Target-Item ausgeschlossen wird
unpopular_items_list = [
    row["ISBN"] for row in unpopular_items_negative.collect() if row["ISBN"] != target_item
]

print("Top 2 unpopuläre Items (ISBN), basierend auf unteren Quartil der Bewertungen, ohne das Target-Item:", unpopular_items_list)

In [ ]:
# Filtere Zeilen, die einen nicht-leeren Publisher haben
books_with_publishers_df = ratings.filter(F.col("Publisher").isNotNull() & (F.col("Publisher") != "")).select("ISBN", "Publisher")

# Funktion zum Aufteilen der Publisher und Erstellen eines Paares (publisher, ISBN)
def split_publishers(row):
    isbn = row["ISBN"]
    publishers = row["Publisher"].split("|")
    return [(publisher, isbn) for publisher in publishers]

# RDD erstellen, indem die Bücher aufgeteilt werden und dann nach Publisher gruppiert werden
books_by_publisher_rdd = books_with_publishers_df.rdd.flatMap(split_publishers).groupByKey()

# Konvertiere das RDD in ein Dictionary, um schnellen Zugriff auf ISBNs nach Publisher zu ermöglichen
books_by_publisher = books_by_publisher_rdd.mapValues(list).collectAsMap()

# Ziel-Item abfragen
target_item_publishers = books_with_publishers_df.filter(F.col("ISBN") == target_item).select("Publisher").collect()[0]["Publisher"].split('|')

# Liste aller Kandidaten für Segment-Items erstellen
segment_books_candidates = []

for publisher in target_item_publishers:
    if publisher in books_by_publisher:
        for book in books_by_publisher[publisher]:
            # Nur hinzufügen, wenn das Segment-Item nicht das Target-Item ist und nicht bereits vorhanden ist
            if book != target_item and book not in segment_books_candidates:
                segment_books_candidates.append(book)

# Zufällige Auswahl von 2 Segment-Items aus den Kandidaten
segment_books = random.sample(segment_books_candidates, 2) if len(segment_books_candidates) > 2 else segment_books_candidates

print(segment_books_candidates)

# Ausgabe der ausgewählten Segment-Items
print(segment_books)

In [ ]:
# Funktion zur Generierung von Filler-Ratings nach Normalverteilung
def generate_filler_rating(mean_rating):
    # Generiere eine Bewertung nach der Normalverteilung basierend auf dem übergebenen Mittelwert
    rating = np.random.normal(mean_rating, stddev_rating)
    # Begrenze das Rating auf die Skala von 1 bis 10
    rating = max(0, min(10, rating))
    # Runde das Rating auf die nächste ganze Zahl (1, 2, ..., 10)
    return round(rating)

In [ ]:
# Angriffstyp auswählen und validieren
attack_type = input("Wähle Angriffstyp ('reverse_bandwagon', 'random_nuke', 'average_nuke' oder 'segment_nuke'): ").lower()
while attack_type not in ['reverse_bandwagon', 'random_nuke', 'average_nuke', 'segment_nuke']:
    print("Ungültige Eingabe! Bitte wähle entweder 'reverse_bandwagon', 'random_nuke', 'average_nuke' oder 'segment_nuke'.")
    attack_type = input("Wähle Angriffstyp ('reverse_bandwagon', 'random_nuke', 'average_nuke' oder 'segment_nuke'): ").lower()

In [ ]:
# Angriffsart anzeigen
print(attack_type)

In [ ]:
# Gruppiere nach 'userId' und zähle die Ratings
ratings_per_user = ratings.groupBy("userId").agg(count("rating").alias("num_ratings"))

# Berechne den Durchschnitt der Ratings pro Benutzer
average_ratings_per_user = ratings_per_user.agg(avg("num_ratings")).collect()[0][0]

# Runde den Durchschnitt und ziehe 1 für das Target-Item ab
rounded_average = round(average_ratings_per_user - 1)

# Ausgabe der durchschnittlichen Anzahl an Ratings pro Profil abzüglich von 1
print("Durchschnittliche Anzahl an Ratings pro Profil (gerundet, nach Abzug von 1):", rounded_average)

In [ ]:
# Funktion zur Generierung einer Profilgröße basierend auf einer Normalverteilung
def generate_attack_profile_size(average):
    # Generiere eine Anzahl von Bewertungen basierend auf einer Normalverteilung
    std_dev = 0.5 * average  # Standardabweichung = 50% des Durchschnitts
    profile_size = int(np.random.normal(average, std_dev))

    # Stelle sicher, dass die Profillänge mindestens 1 ist
    profile_size = max(profile_size, 1)
    
    return profile_size

In [ ]:
# Funktion zum Erstellen von Angriffs-Bewertungen für jeden neuen User
def generate_attack_ratings(user_id, target_item, unpopular_items_list, all_isbns, attack_type, segment_books):
    ratings_list = []
    
    # Ziel-Item mit der minimalen Bewertung (0) bewerten
    ratings_list.append((user_id, target_item, 0))
    
    # Bestimme die Anzahl der Filler-Items basierend auf der Normalverteilung
    num_fillers = generate_attack_profile_size(rounded_average)
    
    # Reduziere die Anzahl der Filler-Items bei Reverse Bandwagon und Segment Nuke-Angriffen
    if attack_type == 'reverse_bandwagon' or attack_type == 'segment_nuke':
        num_fillers = max(num_fillers - 2, 0)  # Setze Untergrenze auf 0
    
    # Füge unpopuläre Items hinzu bei Reverse Bandwagon
    if attack_type == 'reverse_bandwagon':
        # 2 unpopulärste Items mit 0 bewerten
        for ISBN in unpopular_items_list:
            ratings_list.append((user_id, ISBN, 0))
    
    # Füge Segment-Items hinzu bei Segment Nuke-Angriff
    elif attack_type == 'segment_nuke' and segment_books:
        # 2 Items aus der vorbereiteten Segment-Liste bewerten
        for ISBN in segment_books:
            ratings_list.append((user_id, ISBN, 0))

    # Liste der nicht erlaubten Items (immer das Target-Item)
    not_allowed_items = set([target_item])

    # Füge bei Reverse Bandwagon-Angriff die unpopulären Items hinzu
    if attack_type == 'reverse_bandwagon':
        not_allowed_items.update(unpopular_items_list)
    
    # Füge bei Segment Nuke-Angriff die Segment-Items hinzu
    if attack_type == 'segment_nuke' and segment_books:
        not_allowed_items.update(segment_books)
    
    # Filtere die Filler-Items, sodass sie keine nicht erlaubten Items enthalten
    available_filler_items = [ISBN for ISBN in all_isbns if ISBN not in not_allowed_items]

    # Wähle zufällige Filler-Items aus den verbleibenden Büchern aus
    random_books = random.sample(available_filler_items, num_fillers)
    
    # Filler-Items bewerten
    for ISBN in random_books:
        if attack_type == 'average_nuke':
            mean_rating = item_avg_ratings_dict.get(ISBN, mean_rating_system)
            filler_rating = generate_filler_rating(mean_rating)
        elif attack_type == 'segment_nuke':
            filler_rating = 10  # Bei Segment Nuke-Angriffen werden Filler-Items immer mit der maximalen Bewertung (10) bewertet
        else:
            filler_rating = generate_filler_rating(mean_rating_system)

        ratings_list.append((user_id, ISBN, filler_rating))
    
    return ratings_list

# Liste aller verfügbaren Bücher
all_isbns = ratings.select("ISBN").distinct().rdd.flatMap(lambda x: x).collect()

# Generiere Angriffs-Bewertungen und speichere sie als DataFrame
attack_ratings = []
for i in range(num_attack_profiles):
    user_id = 100000000 + i  # Erstelle eine neue UserID für jeden Angriff (IDs >= 100000000 sind neu bzw. noch nicht vorhanden)
    
    # Erzeuge Angriffsbewertungen
    attack_ratings.extend(generate_attack_ratings(
        user_id, 
        target_item, 
        unpopular_items_list, 
        all_isbns, 
        attack_type, 
        segment_books
    ))

# Erstelle ein DataFrame mit den generierten Angriffsbewertungen
attack_ratings_df = spark.createDataFrame(attack_ratings, ["userId", "ISBN", "rating"])

# Zeige die ersten 10 Einträge des Angriffs-Dataframes
attack_ratings_df.show(10)

In [ ]:
# Anzahl der Einträge im attack_ratings_df
num_entries = attack_ratings_df.count()
print(f"Anzahl der Einträge (Bewertungen) in Angriffsratings: {num_entries}")

In [ ]:
# Angriffsart anzeigen
print(attack_type)

In [ ]:
# Anzahl verschiedener userIds
num_users = attack_ratings_df.select("userId").distinct().count()
print(f"Anzahl verschiedener User IDs in Angriffsratings: {num_users}")

In [ ]:
# Anzahl verschiedener ISBNs
num_books = attack_ratings_df.select("ISBN").distinct().count()
print(f"Anzahl verschiedener ISBNs: {num_books}")

In [ ]:
# Häufigkeit der Ratings im Angriffs-Dataframe
rating_counts = attack_ratings_df.groupBy("rating").count().orderBy("rating")
rating_counts.show()

# Alternativ als Python-Dictionary
rating_counts_dict = {row['rating']: row['count'] for row in rating_counts.collect()}
print(f"Häufigkeit der Bewertungen in Angriffsratings: {rating_counts_dict}")

In [ ]:
# Nur relevante Spalten aus dem ursprünglichen ratings DataFrame auswählen
ratings = ratings.select("userId", "ISBN", "rating")

In [ ]:
# Ursprüngliche Daten und Angriffs-Daten kombinieren
all_ratings = ratings.union(attack_ratings_df)
all_ratings.show(10)

In [ ]:
# Häufigkeit der Ratings im Gesamt-Dataframe
rating_counts2 = all_ratings.groupBy("rating").count().orderBy("rating")
rating_counts2.show()

In [ ]:
# Label-Spalte hinzufügen: 0 für normale User (vorhandene), 1 für Angriffs-User
all_ratings = all_ratings.withColumn("Label", 
                                     when(col("userId") >= 100000000, 1)
                                     .otherwise(0))

# Überprüfen des finalen Datensatzes
all_ratings.show(10)

In [ ]:
# Variable mit Namen des Datensatzes
datensatz = "BookCrossing"

In [ ]:
# Dynamischer Dateiname basierend auf Datensatz, Angriffsart und Angriffsgröße
dateiname = f"all_ratings_{datensatz}_{attack_type}_{percentage_of_attacks}.csv"

# Speichern des DataFrames mit dem dynamischen Dateinamen
all_ratings.coalesce(1).write.csv(dateiname, header=True)

In [ ]:
# Filter auf userId >= 100000000 (Angreifer)
filtered_ratings = all_ratings.filter(col("userId") >= 100000000)

# Anzeigen der gefilterten Zeilen
filtered_ratings.show(10)

In [ ]:
# Filter auf userId >= 100000000
filtered_ratings2 = all_ratings.filter(col("userId") == 100000000)

# Anzeigen der Bewertungen von Nutzer mit userId 100000000
filtered_ratings2.show(10)

# Häufigkeit der Ratings bei User 100000000
rating_counts2 = filtered_ratings2.groupBy("rating").count().orderBy("rating")
rating_counts2.show()

# Alternativ als Python-Dictionary
rating_counts_dict2 = {row['rating']: row['count'] for row in rating_counts2.collect()}
print(f"Häufigkeit der Bewertungen: {rating_counts_dict2}")

In [ ]:
# Berechne den Durchschnitt der Bewertungen für Nutzer mit userId 100000000
average_rating_user = attack_ratings_df.filter(col("userId") == 100000000).agg(avg("rating")).collect()[0][0]

print(f"Der Durchschnitt der Bewertungen von User 100000000 ist: {average_rating_user}")

In [ ]:
# Berechne die neue durchschnittliche Bewertung im Gesamt-Dataframe nach einem Angriff
mean_rating2 = all_ratings.agg({"rating": "avg"}).collect()[0][0]
print(mean_rating2)

In [ ]:
# Anzahl aller Ratings im Angriffs-DataFrame
total_ratings_count_attack = attack_ratings_df.count()

# Ausgabe der Anzahl der Ratings
print("Anzahl aller Angriffsratings:", total_ratings_count_attack)

In [ ]:
# Anzahl aller Ratings im Gesamt-Dataframe
total_ratings_count = all_ratings.count()

# Ausgabe der Anzahl der Ratings
print("Anzahl aller Ratings:", total_ratings_count)

In [ ]:
# Definiere die User-IDs für die Abfrage
user_ids_mio = [i for i in range(100000000, 100000021)]  # User von 100000000 bis 100000020

# Filtere den Datensatz für die gewünschten User und zähle die Ratings pro User
ratings_count_mio = all_ratings.filter(F.col("userId").isin(user_ids_mio)) \
                       .groupBy("userId") \
                       .agg(F.count("rating").alias("num_ratings")) \
                       .orderBy("userId")

# Zeige die Anzahl der Ratings für die angegebenen User
ratings_count_mio.show()

In [ ]:
# Beende die Spark-Session
spark.stop()